In [ ]:
from pprint import pprint

import json
import subprocess
from pathlib import Path

import numpy as np
import cv2
import cv2.aruco as aruco


In [ ]:
from scipy import interpolate


def fill_nan(A):
    """
    interpolate to fill nan values
    """
    inds = np.arange(A.shape[0])
    good = np.where(np.isfinite(A))
    f = interpolate.interp1d(inds[good], A[good], bounds_error=False)
    B = np.where(np.isfinite(A), A, f(inds))
    return B

In [ ]:
data_dir = Path("drive/data")
baseline_dir = data_dir / "baseline"
ours_dir = data_dir / "ours"
base_data_dirs = list(baseline_dir.glob("*"))
ours_data_dirs = list(ours_dir.glob("*"))


## make visualization


In [ ]:
for data_dir in base_data_dirs:
    cmd = f"python3 distance_estimation/distance_object_analysis.py -p {str(data_dir)}"
    result = subprocess.check_output(cmd, shell=True)


In [ ]:
for data_dir in ours_data_dirs:
    cmd = f"python3 distance_estimation/distance_object_analysis.py -p {str(data_dir)}"
    result = subprocess.check_output(cmd, shell=True)


## get distance & rect_size from video


In [ ]:
width = 854
height = 480
marker_length = 0.1

with open("distance_estimation/calibrationValues0.json") as f:
    cal_vals = json.load(f)
cam_mtx = np.array(cal_vals["camera_matrix"])
distor_factor = np.array(cal_vals["dist_coeff"])
aruco_dict = aruco.Dictionary_get(cv2.aruco.DICT_4X4_250)


### baseline


In [ ]:
def get_distance(frame):
    corners, ids, _ = aruco.detectMarkers(frame, aruco_dict)
    rvec, tvec, _ = aruco.estimatePoseSingleMarkers(
        corners, marker_length, cam_mtx, distor_factor
    )
    if ids is not None:
        distance = tvec[0][0][2] * 100
    else:
        distance = np.nan
    return distance

In [77]:
def get_size_from_object_data(data):
    if len(data) >= 1:
        _, _, object = data[0]
        xmin, ymin, xmax, ymax = object
        return (xmax - xmin) * (ymax - ymin)
    else:
        return 0


In [84]:
def get_stop_start_frame(object_data):
    new_obj_data = {}
    for d in object_data:
        frame_cnt, obj = d.values()
        new_obj_data[frame_cnt] = obj
        if get_size_from_object_data(obj) >= 2000:
            return frame_cnt


In [90]:
distance_lists = []
stop_dist = []
end_dist = []
for data_dir in base_data_dirs:
    distance_list = []
    video_path = data_dir / "video.avi"
    object_data_path = data_dir / "objects.json"
    with open(object_data_path) as f:
        object_data = json.load(f)

    stop_frame = get_stop_start_frame(object_data)

    cap = cv2.VideoCapture(str(video_path))
    frame_cnt = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if ret:
            distance = get_distance(frame)
            distance_list.append(distance)

            frame_cnt += 1
        else:
            break

    distance_list = np.array(distance_list)
    distance_list = fill_nan(distance_list)

    stop_dist.append(distance_list[stop_frame])
    end_dist.append(distance_list[-1])
    # print(distance_list[stop_frame], distance_list[-1])

    distance_lists.append(distance_list)

print(
    f"---- stop distance analysis ----\n"
    f"mean: {np.mean(stop_dist)}\n"
    f"min: {min(stop_dist)}\n"
    f"max: {max(stop_dist)}\n"
)
print(
    f"---- end distance analysis ----\n"
    f"mean: {np.mean(end_dist)}\n"
    f"min: {min(end_dist)}\n"
    f"max: {max(end_dist)}\n"
)


---- stop distance analysis ----
mean: 108.92292972685314
min: 72.82905103605648
max: 161.28349599737314

---- end distance analysis ----
mean: 94.98008964836689
min: 58.58506583270052
max: 147.5446138515846



### ours

In [91]:
distance_lists = []
stop_dist = []
end_dist = []
for data_dir in ours_data_dirs:
    distance_list = []
    video_path = data_dir / "video.avi"
    object_data_path = data_dir / "objects.json"
    with open(object_data_path) as f:
        object_data = json.load(f)
        
    stop_frame = get_stop_start_frame(object_data)

    cap = cv2.VideoCapture(str(video_path))
    frame_cnt = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if ret:
            distance = get_distance(frame)
            distance_list.append(distance)

            frame_cnt += 1
        else:
            break

    distance_list = np.array(distance_list)
    distance_list = fill_nan(distance_list)
    
    stop_dist.append(distance_list[stop_frame])
    end_dist.append(distance_list[-1])
    # print(distance_list[stop_frame], distance_list[-1])

    distance_lists.append(distance_list)
    

print(
    f"---- stop distance analysis ----\n"
    f"mean: {np.mean(stop_dist)}\n"
    f"min: {min(stop_dist)}\n"
    f"max: {max(stop_dist)}\n"
)
print(
    f"---- end distance analysis ----\n"
    f"mean: {np.mean(end_dist)}\n"
    f"min: {min(end_dist)}\n"
    f"max: {max(end_dist)}\n"
)


---- stop distance analysis ----
mean: 178.32329692756724
min: 159.5264410845568
max: 205.79469814975045

---- end distance analysis ----
mean: 160.50334720000572
min: 143.1562351811378
max: 188.01301491293157

